# 1. 준비단계 - 모듈 설치, 데이터셋 받기 및 확인

---





## 기본 모듈 설치

## 학습 시

## 기본 모듈 임포트

In [ ]:
import os

import json
import numpy as np
import pandas as pd
import re
import string
from collections import Counter
from tqdm.notebook import tqdm

import torch
from transformers import (
    pipeline,
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer
)
from datasets import load_dataset, Dataset, DatasetDict
from accelerate import Accelerator
from peft import LoraConfig, PeftConfig, PeftModel, AutoPeftModelForCausalLM
from trl import SFTTrainer
from peft import get_peft_model, PromptTuningInit, PromptTuningConfig, TaskType, PeftType
from unsloth import FastLanguageModel
from unsloth import is_bfloat16_supported
import wandb

## 입력파일 확인

In [ ]:
VAL_DATA_COUNT = 100 #validation data는 몇개로 할것인지 선택
MAX_TOKEN = 1100 #데이터를 토크나이저에 넣어 나온 토큰의 길이 중 최대를 파악한 뒤 입력해주면 됨.
MAX_TRAIN_COUNT = 30000 #데이터 몇개를 사용할 건지 선택

In [ ]:
file_path = './data/train.csv'

train_data = pd.read_csv(file_path)

train_data = train_data.sample(frac=1).reset_index(drop=True) #무작위 섞고, 인덱스 0부터로 초기화

val_data=train_data[:VAL_DATA_COUNT]
train_data = train_data[VAL_DATA_COUNT:].reset_index(drop=True)

val_label_df = val_data[['question', 'answer']]

train_data.head(2)




# 2. 모델 및 부가 함수 정의

---



## 토큰으로 로그인하기

In [ ]:
from huggingface_hub import login

login(token="YOUR_HUGGINGFACE_TOKEN")

## 1. 데이터 정제 및 데이터셋 설정

### 데이터 셋 토크나이즈하기

In [ ]:
model_id = "beomi/Llama-3-KoEn-8B-Instruct-preview"

tokenizer = AutoTokenizer.from_pretrained(model_id)

# tokenizer.eos_token : 문장의 끝을 나타내는 토큰. 패딩 토큰을 문장의 끝 토큰으로 설정하여 문장의 패딩을 수행할 것임을 의미
tokenizer.pad_token = tokenizer.eos_token

# 패딩을 적용할 쪽을 오른쪽으로 설정. 토크나이저가 패딩 토큰을 문장의 끝에 추가할 것임을 나타냄. 즉, 오른쪽에 패딩을 적용
tokenizer.padding_side = "right"

#### 준비과정 1 - 데이터셋 context와 question으로 합친 문자열을 토큰화하기

In [ ]:
# 데이터셋 학습에 알맞게 변경하기 => context와 question 합치기
def preprocess_function(context, question, answer):
    inputs = [
        {"role": "system", "content": "A chat between a curious user and an artificial intelligence assistant. The assistant gives short answers to the user's questions."},
        {"role": "user", "content": f"Read the context. Answer shortly like one word. Context: {context}\nQuestion: {question}"},
        {"role": "assistant", "content": f"{answer}"},
    ]
    model_inputs = tokenizer.apply_chat_template(inputs, tokenize=False, add_generation_prompt=False)
    model_inputs = tokenizer(model_inputs)
    
    return model_inputs

tokenized_train_datasets = train_data.apply(lambda row: preprocess_function(row['context'], row['question'], row['answer']), axis=1)
tokenized_valid_datasets = val_data.apply(lambda row: preprocess_function(row['context'], row['question'], row['answer']), axis=1)


#### 준비과정 2 - 데이터별 토큰 개수 분포 확인

In [ ]:
count = {
    '1000 under' : 0,
    '1000~1100' : 0,
    '1100~1200' : 0,
    '1200~1300' : 0,
    '1300~1400' : 0,
    '1400~1500' : 0,
    '1500~1600' : 0,
    '1600~1700' : 0,
    '1700~1800' : 0,
    '1800~1900' : 0,
    '1900~2000' : 0,
    '2000~2100' : 0,
    '2100~2200' : 0,
    '2200~2300' : 0,
    '2300~2400' : 0,
}
for item in tokenized_train_datasets:
    temp = len(item['input_ids'])
    if (temp < 1000):
        count['1000 under']+=1
    elif (temp >=1000 and temp<1100):
        count['1000~1100']+=1
    elif (temp >=1100 and temp<1200):
        count['1100~1200']+=1
    elif (temp >=1200 and temp<1300):
        count['1200~1300']+=1
    elif (temp >=1300 and temp<1400):
        count['1300~1400']+=1
    elif (temp >=1400 and temp<1500):
        count['1400~1500']+=1
    elif (temp >=1500 and temp<1600):
        count['1500~1600']+=1
    elif (temp >=1600 and temp<1700):
        count['1600~1700']+=1
    elif (temp >=1700 and temp<1800):
        count['1700~1800']+=1
    elif (temp >=1800 and temp<1900):
        count['1800~1900']+=1
    elif (temp >=1900 and temp<2000):
        count['1900~2000']+=1
    elif (temp >=2000 and temp<2100):
        count['2000~2100']+=1
    elif (temp >=2100 and temp<2200):
        count['2100~2200']+=1
    elif (temp >=2200 and temp<2300):
        count['2200~2300']+=1
    elif (temp >=2300 and temp<2400):
        count['2300~2400']+=1
for key in count:
    print(f"{key} : {count[key]}")

#### 준비과정 3 - 최대 토큰 개수이하로 선별하기

In [ ]:
filtered_tokenized_train_datasets = pd.Series([])
index = 0

for item in tokenized_train_datasets:
    if filtered_tokenized_train_datasets.shape[0] < MAX_TRAIN_COUNT and len(item['input_ids']) <= MAX_TOKEN:
        item = tokenizer.decode(item['input_ids'])
        item = tokenizer(item, max_length=MAX_TOKEN, truncation=True, padding='max_length')
        filtered_tokenized_train_datasets[index] = item
        index+=1

filtered_tokenized_train_datasets

### 학습을 위해 Dataset으로 설정 

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]).to("cuda") for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings['input_ids'])

# tokenized_datasets를 리스트에서 별도의 키로 변환
train_encodings = {
    'input_ids': [item['input_ids'] for item in filtered_tokenized_train_datasets],
    'attention_mask': [item['attention_mask'] for item in filtered_tokenized_train_datasets],
}
val_encodings = {
    'input_ids': [item['input_ids'] for item in tokenized_valid_datasets],
    'attention_mask': [item['attention_mask'] for item in tokenized_valid_datasets],
}

train_datasets = CustomDataset(train_encodings)
val_datasets = CustomDataset(val_encodings)



## 2. 학습하기

### lora(Unsloth) 설정

In [ ]:
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install -qqq flash-attn
    # 고성능 Attention인 flash attention 2 을 사용
    attn_implementation = "flash_attention_2"
    # 데이터 타입을 bfloat16으로 설정해준다.
    # bfloat16은 메모리 사용량을 줄이면서도 계산의 정확성을 유지할 수 있는 데이터 타입이다.

    torch_dtype = torch.bfloat16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float16

In [ ]:
# lora 설정
use_dora = False
lora_alpha=64
lora_dropout=0.2
r=32

peft_params = LoraConfig(
    use_dora=use_dora,  #dora 사용
    lora_alpha=lora_alpha,  # attention 계산에 사용되는 가중치 행렬의 크기를 나타냄. 일반적으로 큰 값일수록 모델의 표현력이 높아지지만, 계산 비용이 증가
    lora_dropout=lora_dropout,  # attention 계산 시에 적용되는 드롭아웃의 확률을 지정. 드롭아웃은 모델의 과적합을 방지하고 일반화 성능을 향상시키는 데 사용
    r=r,  # attention 계산에 사용되는 쿼리(query), 키(key), 밸류(value)의 차원을 결정. 이 값은 일반적으로 lora_alpha와 관련이 있으며, 주로 모델의 표현력과 계산 비용을 조정하는 데 사용
    target_modules=[ #모델의 어텐션 중 어느곳에 lora 어댑터를 붙일지를 설정
    "q_proj",
    "up_proj",
    "o_proj",
    "k_proj",
    "down_proj",
    "gate_proj",
    "v_proj"],
    bias="none",  # attention 계산의 편향(bias)을 지정. 여기서는 "none"으로 설정되어 있으므로, 어텐션 계산에 편향이 추가되지 않음.
    task_type="CAUSAL_LM",  # 모델이 수행할 작업 유형. 여기서는 모델이 인과 언어 모델링(causal language modeling) 작업을 수행. 이는 주어진 이전 토큰들을 기반으로 다음 토큰을 예측하는 작업을 의미
)

### 사전학습 모델 불러오기 using Unsloth

In [ ]:
model, t = FastLanguageModel.from_pretrained(
    model_name = model_id,
    max_seq_length = MAX_TOKEN,
    dtype = torch_dtype,
    device_map = 0,
    load_in_4bit=True,
    attn_implementation = attn_implementation,
)

model = FastLanguageModel.get_peft_model(
    model,
     use_dora=use_dora,  #dora 사용 여부
    lora_alpha=lora_alpha,  # attention 계산에 사용되는 가중치 행렬의 크기를 나타냄. 일반적으로 큰 값일수록 모델의 표현력이 높아지지만, 계산 비용이 증가
    lora_dropout=lora_dropout,  # attention 계산 시에 적용되는 드롭아웃의 확률을 지정. 드롭아웃은 모델의 과적합을 방지하고 일반화 성능을 향상시키는 데 사용
    r=r,  # attention 계산에 사용되는 쿼리(query), 키(key), 밸류(value)의 차원을 결정. 이 값은 일반적으로 lora_alpha와 관련이 있으며, 주로 모델의 표현력과 계산 비용을 조정하는 데 사용
    bias="none",  # attention 계산의 편향(bias)을 지정. 여기서는 "none"으로 설정되어 있으므로, 어텐션 계산에 편향이 추가되지 않음.
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    max_seq_length = MAX_TOKEN,
)
model.print_trainable_parameters()

In [ ]:
model.config.use_cache = False  # 모델이 추론 시 캐시를 사용하지 않도록 함. 캐시를 사용하면 모델의 속도가 빨라질 수 있지만, 여기서는 캐시를 사용하지 않도록 설정
model.config.pretraining_tp = 1  # 모델의 pretraining tensor parallelism(사전 학습 시 텐서 병렬화) 설정을 1로 설정. 이는 병렬화를 사용하지 않도록 설정한 것

### Training Argument 설정(하이퍼 파라미터 설정)

In [ ]:
proj_name = 'llama3_unsloth_qlora_tuning'

wandb.init(project=proj_name)
training_params = TrainingArguments(
    run_name = proj_name,
    output_dir="./checkpoint",  # 학습 결과를 저장할 디렉토리 경로
    num_train_epochs=10,  # 문제지를 몇 번 돌릴 것인지
    per_device_train_batch_size = 4,
    per_device_eval_batch_size = 4,
    # auto_find_batch_size=True, #batch사이즈 잘못 바꾸면 cuda out of memory 에러나므로 그냥 두기
    # gradient_accumulation_steps=4,  # 그래디언트 누적 단계 수. 배치 크기를 사용하면 GPU 메모리 부족으로 배치를 처리할 수 없을 수 있으므로, 그래디언트 누적을 통해 이를 해결
    optim="adamw_8bit",  # 최적화 알고리즘.
    evaluation_strategy = "steps",
    neftune_noise_alpha=15, #NEFTune 사용 - 임베딩 시 노이즈를 섞어 학습시킴
    do_eval = True, #Whether to run evaluation on the validation set or not
    save_steps=50,  # 중간 체크포인트 저장 간격. 모델의 중간 상태를 저장하여 학습 중에 중단되었을 때 학습을 재개
    logging_steps=25,  # 학습 로그 출력 간격. 학습 중간에 로그를 출력하여 모델의 학습 진행 상황을 모니터링
    learning_rate=2e-4,  # 초기 학습률. 학습률은 모델이 가중치를 업데이트하는 속도를 결정
    weight_decay=0.003,  # 가중치 감쇠 정도. 중치 감쇠는 모델의 복잡도를 줄이고 과적합을 방지하는 데 도움
    fp16 = True if torch_dtype != torch.bfloat16 else False,  # mixed precision 학습 사용 여부 (16비트/32비트). True로 설정하면 16비트 부동소수점 형식을 사용하여 학습 속도를 높임
    bf16 = True if torch_dtype == torch.bfloat16 else False,  # bfloat16 precision 학습 사용 여부 (16비트/32비트)
    max_grad_norm=0.3,  # 그래디언트 클리핑을 위한 최대 그래디언트 노름. 그래디언트 클리핑은 그래디언트 폭주를 방지하고 안정적인 학습을 위해 사용
    max_steps=-1,  # 최대 학습 스텝 수 (-1이면 무제한). 학습 스텝 수가 지정된 값에 도달하면 학습이 중단
    warmup_ratio=0.03,  # 학습률 스케줄러의 초기 웜업 비율. 웜업 단계에서는 초기 학습률을 점진적으로 증가시켜 안정적인 학습을 유도
    group_by_length=True,  # 그룹 당 장치 개수. 분산 학습을 위해 사용
    lr_scheduler_type="cosine",  # 학습률 스케줄러 타입. 여기서는 "constant"로 설정되어 있으며, 학습률을 일정한 상수 값으로 유지
    # per_device_group_size=True,  # 그룹 당 장치 개수. 분산 학습을 위해 사용
    # learning_rate_scheduler_type="constant",  # 학습률 스케줄러 타입. 여기서는 "constant"로 설정되어 있으며, 학습률을 일정한 상수 값으로 유지
    report_to="wandb"  # 학습 결과 보고서를 전송할 위치. TensorBoard를 사용하여 학습 과정을 모니터링
)

In [ ]:
trainer = SFTTrainer(
    model=model,  # 학습할 모델 (사전 학습된 모델)
    train_dataset=train_datasets,  # 학습 데이터셋 (새로 학습할 추가 데이터셋)
    eval_dataset = val_datasets,
    dataset_text_field="input_ids",  # 데이터셋에서 텍스트 필드의 이름
    max_seq_length=MAX_TOKEN,  # 최대 시퀀스 길이
    tokenizer=tokenizer,  # 토크나이저
    args=training_params,  # 학습 인자들 (파라미터 설정 넣어주는 부분?)
    peft_config=peft_params,  # PEFT 설정
    packing=False,  # 패킹 사용 여부
)

In [ ]:
trainer.train()

## 3. 파인튜닝한 모델 저장

In [ ]:
trainer.model.save_pretrained("./"+proj_name)